# Redes neuronales hacia adelante

En esta libreta vamos a realizar las funciones necesarias para entrenar y predecir utilizando una red neuronal hacia adelante multicapa, con función de activación logística en *todas* las neuronas de las capas ocultas. Esta libreta no pretende sustituir a las explicaciones en clase o a unas notas sobre redes neuronales. Aqui se asume que ustedes ya tienen una idea general de las redes neuronales, que comprenden el algoritmo de *backpropagation* tal como lo desarrollamos en clase. En esta libreta *solamente* nos vamos a centrar en los aspectos de implementación.

Para empezar esta libreta, necesitaremos algunas de las funciones que ya programamos en libretas pasadas, las cuales adjuntamos a continuación.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cPickle
from scipy.optimize import minimize
from IPython.display import Image 


def obtiene_medias_desviaciones(x):
    """
    Obtiene las medias y las desviaciones estandar atributo a atributo.
    
    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @return: medias, desviaciones donde ambos son ndarrays de dimensiones (n,) con las medias y las desviaciones 
             estandar respectivamente.
    
    """
    return x.mean(axis=0), x.std(axis=0)

def normaliza(x, medias, desviaciones):
    """
    Normaliza los datos x

    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @param medias: ndarray de dimensiones (n,) con las medias con las que se normalizará
    @param desviaciones: ndarray de dimensiones (n,) con las desviaciones con las que se normalizará
    
    @return: x_norm un ndarray de las mismas dimensiones de x pero normalizado
    
    """
    return (x - medias) / desviaciones


def logistica(z):
    """
    Calcula la función logística para cada elemento de z
    
    @param z: un ndarray
    @return: un ndarray de las mismas dimensiones que z
    """
    return 1 / (1 + np.exp(-z))

def softmax(z):
    """
    Calculo de la regresión softmax
    
    @param z: ndarray de dimensión (T, K) donde z[i, :] es el vector de aportes lineales de el objeto i    
    @return: un ndarray de dimensión (T, K) donde cada columna es el calculo softmax de su respectivo vector de entrada.
    
    """
    y_hat = np.exp(z)
    return y_hat / y_hat.sum(axis=1).reshape(-1,1)


## 1. Especificando una red neuronal

Primero, para poder hacer una red neuronal, tenemos que determinar cierta información. Por el momento, y para efecto de la libreta, vamos a mantenernos en un esquema estructurado/funcional, y más adelante vamos a generalizar la mayoría de los métodos utilizados en forma de objetos.

La información importante que debemos de tener en cuenta cuando hacemos un sistema de redes neuronales es:

- Cuantas capas de neuronas tiene la red neuronal, $L$.
- Cuantas neuronas va a tener cada capa $[n_0, n_1, \ldots, n_L]$, donde $n_0$ es el número de entradas y $n_L$ el número de salidas.
- Cual es el tipo de salida de mi red neuronal (lineal, logística o softmax)
- Los valores con los que se normalizan los datos de entrada a la red neuronal (para el aprendizaje en una red neuronal es muy importante que los valores de entrada estén normalizados).

Una vez que se establecen estos valores, es necesario generar una lista de matrices $[W^{(1)}, \ldots, W^{(L)}]$ donde $W^{(l)}$ es una matriz de dimensiones $(n_l, n_{l-1} + 1)$ de parámetros o pesos. Como vimos en clase, si se inicializan los valores de las entradas de $W^{(l)}$ iguales, es equivalente a tener una sola neurona en esa capa, por lo que es necesario que estos valores sean diferentes. 

En clase vimos que, para efectos de un mejor aprendizaje, es importante que los valores de entrada se encientren en la zona donde casua más variacion la función logística. Para esto, se espera que en general la suma de los pesos multiplicados por las entradas correspondientes a la capa se encuentren en el rango de $(-1, 1)$. Si asumimos que las entradas a cada neurona están normalizadas (esto es, entre 0 y 1), entonces los pesos deberían ser valores entre $(-\sqrt{n_{l-1}}, \sqrt{n_{l-1}})$ con el fin que la suma se encuentre en la región donde más cambios ocurren en la función logística. 

Vamos a generar y guardar esta información en un diccionario (junto con el resto de la información que requeriramos para tener una red neuronal completamente definida. Al principio los valores de normalización no cuentan ya que estos se deben inicializar al comienzo del aprendizaje.

#### Ejercicio 1. Completa el código de la función de inicialización para los pesos de las matrices de pesos (10 puntos).

In [ ]:
def inicializa_red_neuronal(capas, neuronas_por_capa, tipo):
    """
    Inicializa una red neuronal como un diccionario de datos
    
    @param capas: Un número entero con el número total de capas. Minimo 3 (una de entrada, una oculta, una de salida).
    @param neuronas_por_capa: Una lista de enteros donde el primer elemento es el número de entradas
                              y el último el número de salidas, mientras que los intermedios son
                              el númerode neuronas en cada capa oculta.
    @param tipo: Un string entre {'lineal', 'logistica', 'softmax'} con el tipo de salida de la red.
    
    @return: Un diccionario `rn` tal que
             - rn['capas'] = capas
             - rn['nxc'] = neuronas_por_capas
             - rn['tipo'] = tipo
             - rn['W'] = lista de matrices de parámetros
             - rn['medias'] = lista de medias de cada atributo (se inicializan com puros 0)
             - rn['std'] = lista de desviaciones estandard de cada atributo (se inicializan con puros 1)
             
    """
    if capas != len(neuronas_por_capa):
        raise ValueError('El número de capas no corresponde con la lista de las neuronas por capa')
    rn = {'capas': capas, 'nxc': neuronas_por_capa, 'tipo': tipo}
    rn['medias'] = np.zeros(neuronas_por_capa[0])
    rn['std'] = np.ones(neuronas_por_capa[0])
    
    rn['W'] = []
    for (nla, nl) in zip(neuronas_por_capa[:-1], neuronas_por_capa[1:]):
        rn['W'].append(inicializa_W(nla, nl))

    return rn

def inicializa_W( n_lm1, n_l):
    """
    Inicializa una matriz de valores aleatorios W
    
    @param n_lm1: número de neuronas en la capa l-1 (entero)
    @param n_l: número de neuronas en la capa l (entero)
    
    @return: Un ndarray de dimensión (n_l, n_lm1 + 1) donde las entradas son número aleatorios
             entre -sqrt(n_lm1) y sqrt(n_lm1)
             
    """
    #------------------------------------------------------------------------
    # Agregua aqui tu código
    
    #-------------------------------------------------------------------------

def test_inicializa_W():
    #Vamos a hacer 1000 pruebas aleatorias que nos aseguremos que se cumpleen con las especificaciones
    for _ in range(1000):
        n0 = np.random.randint(1, 20)
        n1 = np.random.randint(1, 20)
        W = inicializa_W( n0, n1)
        assert W.shape == (n1, n0 + 1)  # Las dimensiones son correctas
        assert W.max() < np.sqrt(n0)    # La cota máxima se respeta
        assert W.min() > -np.sqrt(n0)   # La cota mínima se respeta
        assert np.abs(W).sum() > 0      # No estamos inicializando a 0
    return "Paso la prueba"

print test_inicializa_W()
    

Así, si tenemos una red neuronal, la información contenida en el diccionario es toda la información específica que se necesita para la predicción, el aprendizaje, o el reaprendizaje de una red ya especificada. 

Como entrenar una red es algo lento y tedioso, y normalmente cuando hacemos un método de aprendizaje, lo que queremos es poder utilizarlo después para predecir un conjunto de datos no etiquetados previamente, es normal que guardemos en un archivo la información específica a la red neuronal, y despues la recuperemos en otra sesión, otro día, o en otra computadora para hacer la predicción.

Una manera de guardar datos, funciones y objectos de Python en disco es utilizando el módulo ``pickle`` (o su versión compilada para mayor velocidad ``cPickle``). Este modulo permite guardar una serie de objetos de python en forma secuencial en un archivo binario, y luego recuperarlos. Notese que este métdo es diferente a ``np.load`` y ``np.savez``, ya que estos solo permiten guardar (y recuperar) una serie de ndarrays únicamente. 

Vamos entonces a hacer dos funciones muy simples ``guarda_objeto`` y ``carga_objeto``, que utilizaremos más adelante.

In [ ]:
def guarda_objeto(archivo, objeto):
    """
    Guarda un objeto de python en el archivo "archivo". Si el archivo existe, sera reemplazado sin 
    preguntas, al puro estilo mafioso.
    
    @param archivo: string con el nombre de un archivo (aunque no exista)
    @param objeto: Un objeto de python para ser guardado
    
    """
    
    with open(archivo, 'wb') as arch:
        cPickle.dump(objeto, arch, -1)
        arch.close()
        
def carga_objeto(archivo):
    """
    Carga el primer (y se asume que único) objeto contenido en el archivo 'archivo' que debe de ser tipo cPickle.
    
    @param archivo: string con el nombre de un archivo tipo pickle
    @return: El primer objeto dentro del picke
    
    """
    with open(archivo, 'rb') as arch:
        objeto = cPickle.load(arch)
        arch.close()
        return objeto
    
def test_archivo():
    """
    Prueba, para esto vamos a cargar o a leer (o ambas cosas) un objeto en un archivo
    
    Por favor, borrar el archivo cada vez que se pruebe, o para probar la lectura y la escritura
    
    """
    temp = [range(100), 'prueba', True]
    guarda_objeto('prueba.pkl', temp)
    temp =[10, 'no prueba', False]
    otro = carga_objeto('prueba.pkl')
    assert len(otro[0]) == 100
    assert otro[1] == 'prueba'
    assert otro[-1]
    
    return "Pasa la prueba"

print test_archivo()

## 2. Calculando el costo (y por lo tanto feed-forward).

Asumamos que tenemos una red neuronal ya inicializada, y que la vamos a utilizar para calcular el costo de una solución. Como vimos en clase, el costo de la solución depende del tipo de neuronas de salida (que son en realidad la etapa de clasificación). Así, para calcular el costo, es necesario calcular la salida de la red neuronal.

Recordemos que el algoritmo para realizar la alimentación hacia adelante de una red neuronal el algoritmo es el siguiente:

1. Inicializa $a^{(0)}$ asignandole los valores de las entradas

2. Por cada capa $l$ de 1 a $L-1$:

    1. Se calcula el valor de $z^{(l)}$ como $$z^{(l)} = W^{(l)} a_e^{(l-1)},$$ donde $W^{(l)}$ es la 
       matriz de pesos de la capa $l-1$ a la capa $l$, y $a_e{(l-1)}$ es $a^{(l-1)}$ extendida con un 1 al principio 
       el vector.
       
    2. Se calcula $a^{(l)}$ como $$a^{(l)} = g(z^{(l)}),$$ donde $g$ es la función de activación (en nuestro caso hemos 
       decidido utilizar la función logística, pero podríamos tener otras funciones de activación).

3. Se calcula el valor de $z^{(L)}$ como $$z^{(L)} = W^{(L)} a_e^{(L-1)}.$$ 

4. Se calcula $a^{(L)}$ de acuerdo a la función de activación dependiendo del tipo de salida:

    * Si `tipo = 'logistica'` entonces se utiliza la regresión logística (una sola neurona en la capa de salida).
    * Si `tipo = 'lineal'` entonces $a^{(L)} = z^{(L)}$.
    * Si `tipo = 'softmax'` entonces $a^{(L)} = softmax(z^{(L)}).$

5. La salida de la red es $a^{(L)}$.

Aqui hay que tomar en cuenta varias cosas: en primer lugar, la activación de todas las neuronas en todas las capas, y para todos los datos los necesitamos para realizar el algoritmo de *backpropagation*, por lo que se requiere guardarlos. Igualmente, no es eficiente calcular todos los pasos dato por dato, ya que eso lo haría muy, pero muy lento. Así que vamos a aporvechar que los datos vienen en forma de un *ndarray* de numpy y haremos todos los calculos en forma matricial tal como los vimos en clases. 

Sea $X$ la matriz de valores de entrada, entonces $A^{0} = X^T$ es una lista de vectores columna donde cada columna es $a^{(0)}$ para el objeto correspondiente. Así, los calculos se pueden realizar columna por columna, y simplemente
$$ 
Z^{(l)} = W^{(l)}A_e^{(l-1)},
$$
donde $A_e^{(l-1)}$ es $A^{(l-1)}$ agregandole un 1 al inicio de cada vector (o lo que es lo mismo, agregandole un renglon de unos al inicio). Si procedemos de esta forma, entonces es importante recordar que al final $\hat{Y} = (A^{(L)})^T$, ya que para la salida cada renglon es un dato diferente (de acuerdo a nuestra convención desde los otros algoritmos que hamos utilizado), mientras que internamente,para la red neuronal, cada columna proviene de un objeto diferente.

Por último, es importante recordar que la normalización es muy importante para las redes neuronales, especialmente si se utiliza el método de descenso de gradiente, por lo que es importante normalizar los datos antes de que sean utilizados, con la información de normalización que se conoce. Esta normalización, con el fin que sea más rápida se asume se realiza *antes* de ingresar al algoritmo de *feedforward*, por lo que se asume que los datos vienen normalizados.

#### Ejercicio 2. Completa el código de la función de *feedforward* para una red neuronal ya establecida (20 puntos).

In [ ]:
def extendida(matriz):
    """
    Agrega un renglon de unos a una matriz
    
    """
    return np.r_[np.ones((1, matriz.shape[1])), matriz]

def feedforward(X, red_neuronal):
    """
    Calcula la salida estimada para los valores de `X` utilizando red_neuronal
    
    @param X: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    
    @return: `lista_A`, donde `Y_est` es un ndarray de shape (T, k) donde T es el número de objetos y k es 
             el número de salidas (neuronas de salida de la red neuronal).
             
    """    
    # Inicializar A^{(0)}
    # ----------Agragar código aqui -----------------
    
    
    
    
    # Despues vamos a hacer lo propio por cada capa hasta antes de la última
    for wl in red_neuronal['W'][:-1]:         
        # Calcula A_e^{l-1}, Z^{(l)}, A^{l} y agrega a lista_A.        
        # (puede ser todo junto o por partes)
        # ----------Agragar código aqui -----------------
        lista_A.append(logistica(wl.dot(extendida(lista_A[-1]))))
        
    # Calcula A^{L} y agrega a lista_A de acuerdo al tipo de salida
    # ----------Agragar código aqui -------------------------------
    
    
    
    
    
    
    return lista_A
                                      
    
    
def prueba_feedforward():
    """
    Función para validar la función de fedforward 
    
    Se inicializa una red de dos capas (más capa de entrada) y se 
    imponen los pesos. Esto se hizo a mano para las diferentes
    entradas, así que se espera que la función de feedforward de
    cosas similares

    """
    # Inicializa red neuronal
    rn = inicializa_red_neuronal(3, [2, 2, 1], 'lineal')
    
    # Modificamos pesos
    rn['W'][0] = np.array([[0.5, -0.3, -0.7],
                           [-1, 0.2, 0.3]])
    rn['W'][1] = np.array([[0, 0.5, -0.5]])
    
    #Ponemos algunas entradas sencillas
    x = np.array([[0,   0],
                  [1,   1],
                  [-1, -1]])
    
    # Y el valor de A calculado a mano
    A1 = np.array([[0.6225, 0.3775, 0.8176],
                   [0.2689, 0.3775, 0.1824]])
    A2 = np.array([0.17676, 0, 0.3176])
    
    lista_A = feedforward(x, rn)
    assert np.sum(np.abs(lista_A[1] - A1)) <= 0.001
    assert np.sum(np.abs(lista_A[2] - A2)) <= 0.001
    
    #Ahora vamos a ver que pasa si cambiamos la salida
    rn['tipo'] = 'logistica'
    lista_A = feedforward(x, rn)
    A2 = np.array([0.544075, 0.5, 0.5787])

    assert np.sum(np.abs(lista_A[2] - A2)) <= 0.001
    
    return "Paso la prueba"

print prueba_feedforward()
    


Y con  la función de feedforward desarrollada, entonces podemos hacer una función para calcular el costo final, la cual depende de las salidas `Y`, de las salidas estimadas por la red neuronal `Y_est`, y del tipo de salida. Igualmente, para agregar la regularización, es necesario un valor `lammbda` de regularización y los parámetros de la red neuronal (`lista_thetas` de la estructura de datos de la red neuronal).

#### Ejercicio 3: Completa el código de la función de costo (10 puntos).

In [ ]:
def costo(Y, Y_est, tipo):
    """
    Calcula la función de costo de una red neuronal, de acuerdo al tipo de salida 
    
    @param Y: un ndarray de dimension (T, K) con los valores de salida
    @param Y_est: un ndarray de dimension (T, K) con los valores de salida estimados
    @param tipo: Un string que puede ser 'lineal', 'logistica'o 'softmax'
    
    """
    # ----------Agragar código aqui -------------------------------

    
    
    
    
    
def prueba_costo():
    """
    La unidad de prueba de la función costo
    
    """
    Y = np.array([[.1, 2.2, 3.6, 0]]).T
    Y_est = np.array([[.3, 2.0, 3.8, -0.2]]).T
    assert abs(costo(Y, Y_est, 'lineal') - 0.02) < 1e-8
    
    Y = np.array([[1, 0, 1, 0]]).T
    Y_est = np.array([[.9, 0.1, 0.8, 0.2]]).T
    assert abs(costo(Y, Y_est, 'logistica') - 0.164252033486018) < 1e-8
    
    Y = np.array([[1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]).T
    Y_est = np.array([[0.8, 0.1, 0.1], 
                      [0.15, 0.8, 0.05],
                      [0.9, 0.05, 0.05],
                      [0.021, 0.079, 0.9]])
    assert abs(costo(Y, Y_est, 'softmax') - 0.164252033486018) < 1e-8
    
    return 'paso la prueba'

print prueba_costo()
    

## 3. Calculando el gradiente con el algoritmo de *Backpropagation*

Es este apartado se genera el gradiente utilizando simplemente el algoritmo de backpropagarion y se prueba con una red neuronal ya hecha anteriormente. Igualmente se realiza un método para el calculo en forma numérica del gradiente, con fines de comprobación que el algoritmo está funcionando correctamente.

Primero vamos a desarrollar el método de *backpropagation*, el cual es el siguiente:

1. Calcular $\delta^{(L)}$ para todos los datos a partir de $A^{L}$, $Y$ y el tipo de salida

2. Para $l$ de $L-1$ hasta 1 con decrementos de $-1$:

    1. Calcular $\delta^{(l)}$ a partir de $\delta^{(l+1)}$, $W^{(l+1)}$ y $A^{(l)}$
    2. Calcular la derivada en cada uno de los pesos como
       $$
       \frac{\partial J(W)}{w_{ij}^{(l)}} = \frac{1}{card(CA)}\sum_{\forall (x,y) \in CA} a_j^{(l-1)} \delta_i^{(l)}
       $$
       
El desarrollo del algoritmo y los pasos en forma matricial se vieron con detalle en clase, por lo que aqui solo se da un pequeñño bosquejo esperando que se programe correctamente.

#### Ejercicio 4 Completa el código de la función de backpropagation (20 puntos).

In [ ]:
quita_renglon = lambda matriz: matriz[1:,:]

def backpropagation(Y, listaA, red_neuronal):
    """
    Calcula el gradiente de los pesos de una red neuronal
    
    @param Y: ndarray de shape (T, k) donde T es el número de ejemplos y k el número de salidas
    @param listaA: Una lista de activaciones por capas, obtenidas por la función `feedforward`
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal`
    
    @return: `lista_Grad`, una lista del gradiente por peso y por capa
             
    """    
    # Calcula Delta para la primer capa
    delta = (listaA[-1] - Y.T)
    
    #Numero de ejemplos
    N = Y.shape[0]
    
    #Calcula el gradiente de los pesos de la última capa
    lista_Grad = []
    lista_Grad.append(delta.dot(extendida(listaA[-2]).T) / N)
    
    # Despues vamos a hacer lo propio por cada capa hasta antes de la última
    for (Wl, Ala, Alant) in zip(red_neuronal['W'][-1:0:-1], listaA[-2:0:-1], listaA[-3::-1]):         
        # Calcula la delta para la capa anterior.        
        # ----------Agragar código aqui -----------------

        
        
        
        # Calcula el gradiente para los pesos de la capa anterior.        
        # ----------Agragar código aqui -----------------

        
        
        
        
    # Si utilizaste el método append, no olvides de hacer la lista al revés
    lista_Grad.reverse()
    return lista_Grad


El problema es que este tipo de algoritmos a la hora de codificarlos es muy típico que se tengan errores, tanto de concepto como de codificación, por lo que es necesario aplicar pruebas para estar seguros que el algoritmo funciona correctamente. 

Para eso, vamos a programar una forma alternativa de calcular una aproximación del gradiente en forma numérica y altamente ineficiente. Por supuesto que esta función no nos ayuda para utilizarla dentro de un método de optimización pero nos sirve para checar errores que podríamos haber ingresado en nuestro algoritmo.

El método se basa en la idea que

$$
  \left.\frac{\partial f(x)}{\partial x}\right|_{x = x_0} \approx \frac{f(x_0 + \epsilon) - f(x_0 - \epsilon)}{2 \epsilon},  
$$
y esto se realiza para cada uno de los pesos para ir calculando los valores del gradiente. Esto no es nada eficiente pero es un método para poder validar el algoritmo de *backpropagation*, Recuerda que mientras más complejo, y más sensible a errores es un algoritmo, mejores deben de ser las técnicas para probar su correcto funcionamiento.

#### Ejercicio 5 Completa el código de la función de gradiente numérico (10 puntos).

In [ ]:
def gradiente_numerico(X, Y, red_neuronal, epsilon):
    """
    Calcula el gradiente numérico para efectos de prueba del algoritmo de backpropagation.
    Este algortimo se realiza expresamente de manera ineficiente pero clara, ya que su
    funcionamiento debe de ser lo más claro posible.
    
    @param X: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param Y: ndarray de shape (T, k) donde k es el número de salidas
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    @param epsilon: Un número flotante positivo, típicamente muy pequeño
 
    @return: Una lista de gradientes de red_neuronal['W'] con la misma estructura y dimensión
    """
    
    # inicializa los gradientes a cero
    rn = red_neuronal['W'][:]
    listaGrad = [np.ones_like(Wl) for Wl in rn]
    
    for (l, Wl) in enumerate(rn):  #  Por cada capa
        for i in range(Wl.shape[0]):  #  Por cada renglon
            for j in range(Wl.shape[1]):  #  Por cada columna
                # -------------------------------------------
                # Insertar código aquí
                # -------------------------------------------

                
                
                
                
                
                
                
                # --------------------------------------------
    return listaGrad           
    

Ahora vamos a hacer una función de prueba utilizando un conjunto de datos reales (o un subconjunto de estos), y lo vamos a hacer para muchas posibles reinicializaciones de pesos. Este código va a servir para corregir ambos algoritmos de calculo de gradiente. Favor de no seguir más allá en la programación de la red neuronal hasta estar seguro que esto funcione correctamente. Para esto vamos a utilizar una base de datos ya conocida, la de dígitos, utilizada en la libreta de regresión softmax.

In [ ]:
# Datos a utilizar para la prueba
# para no tener que estarlos cargando de nuevo cada vez

# Vamos a utilizar un subconkunto de datos y de atributos
# para que pueda funcionar el gradiente numérico en un tiempo 
# aceptable


data = np.load("digitos.npz")
x_prueba = data['X_entrena'][:100,:10]  # Solo 100 datos y 10 parámetros
y_prueba = data['T_entrena'][:100,:]    # Todas las clases de los primeros 100 datos



In [ ]:
def prueba_gradiente(x, y):
    """
    Unidad de prueba de backpropagation
    
    Utiliza la base de datos de digitos, y pueba con varias capas y varios numeros de neuronas
    
    
    """
    n0 = x.shape[1]
    nL = y.shape[1]
    
    for ocultas in [1, 2, 3]: 
        for neuronas in [1, 2]:
            for tipo in ['lineal', 'logistica', 'softmax']:
                for prueba in range(5):
                    # print "Tipo: ", tipo, ", capas ocultas: ", ocultas, ", neuronas en capas ocultas: ", neuronas * n0
                    rn = inicializa_red_neuronal(ocultas + 2, 
                                                 [n0] + (ocultas * [neuronas * n0]) + [nL],
                                                 tipo)
                    listaA = feedforward(x, rn)
                    listaGrad = backpropagation(y, listaA, rn)
                    listaGradnum = gradiente_numerico(x, y, rn, 1e-3)
                    diferencias = [np.abs(G - Gn).max() for (G, Gn) in zip(listaGrad, listaGradnum)]
                    assert max(diferencias) < 1e-3
    return "Paso la prueba"

print prueba_gradiente(x_prueba, y_prueba)


## 4. Aprendizaje con descenso de gradiente

En este apartado se realiza el decenso de gradiente, incluidos

1. Inercia
2. Parada temprana

y por último se prueba en un problema sencillo (el problema de los dígitos).

Para esto vamos primero a desarrollar el algoritmo de descenso de gradiente, tal como lo vimos en clase, esto es, agregandole el término de inercia, para evitar caer en mínimos locales fácilmente.


#### Ejercicio 6 Completa el código de la función de descenso de gradiente con inercia (10 puntos).

In [ ]:
def des_grad(x, y, red_neuronal, alpha=0.1, gamma=0.9, max_epoch=100):
    """
    Descenso de gradiente con inercia, utilizando backpropagation. Se asumen los datos ya normalizados
    
    @param x: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param y: ndarray de shape (T, k) donde k es el número de salidas
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    @param alpha: Paso de aprendizaje, por default 0.1
    @param gamma: Factor de olvido para el computo de la inercia 
    @param max_epoch: Máximo número de epoch (iteraciones) por default 100
    
    @return: True si se hizo el aprendizaje (modifica red_neuronal por referencia)

    """
    lista_Inc = [np.zeros_like(Wl) for Wl in red_neuronal['W']]
    for epoch in xrange(max_epoch):
        lista_A = feedforward(x, red_neuronal)
        lista_Grad = backpropagation(y, lista_A, red_neuronal)
        for (l, Grad) in enumerate(lista_Grad):
            # -------------------------------------------
            # Insertar código aquí
            # -------------------------------------------

                
                
            # --------------------------------------------

    return True

y a partir de esta función, pues se puede construir la funcion necesaria para hacer parada temprana (*early stopping*)

#### Ejercicio 7 Completa el código de la función de parada temprana (10 puntos).

In [ ]:
def early_stop(x, y, red_neuronal, alpha=0.1, gamma=0.9, 
               inter_epoch=100, max_epoch=1000, porcentaje=0.8, criterio=0.1):
    """
    Parada temprana como función envolvente de des_grad

    @param x: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param y: ndarray de shape (T, k) donde k es el número de salidas
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal`
    @param alpha: Paso de aprendizaje, por default 0.1
    @param gamma: Factor de olvido para el computo de la inercia 
    @param inter_epoch: Máximo número de epoch (iteraciones) antes de checar por el sobreaprendizaje por default 100
    @param max_epoch: Máximo número de epoch (iteraciones) por default 1000
    @param porcentaje: Porcentaje de datos usados para aprendizaje (el resto para validación) por default 0.8 (80%).
    @param criterio: Minimo valor para considerar sobreaprendizaje (default 0.1)

    @return: True si se hizo el aprendizaje (modifica red_neuronal por referencia) sin parada temprana,
             si hubo parada temprana devuelve el número de epoch realizados
    
    """
    limite = int(x.shape[0] * porcentaje) 
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    xt, xv = x[indices[:limite], :], x[indices[limite:], :] 
    yt, yv = y[indices[:limite], :], y[indices[limite:], :] 
    
    anterior = 1e10
    for periodos in xrange(max_epoch/inter_epoch):
        # -------------------------------------------
        # Insertar código aquí
        # -------------------------------------------

                
                
        # --------------------------------------------

    return True   

Y por último podemos hacer dos funciones envolventes sencillas, 
una de aprendizaje y otra de predicción, donde ya se incluya todo junto. Estas funciones son provistas.

Tomese en cuenta que la función de aprendizaje toma en cuenta que se empieza el aprendizaje de nada (desde inicializar
la red neuronal), y que contempla reinicios aleatorios (por default 5). Para otras cosas es necesario cambiar las funciones aqui presentadas.

In [ ]:
from copy import deepcopy

def aprende(x, y, tipo, capas_ocultas, neuronas_en_capas_ocultas,
            alpha=0.1, gamma=0.9, inter_epoch=100, max_epoch=1000, 
            porcentaje=0.8, criterio=0.1, reinicios=5):
    """
    @param x: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param y: ndarray de shape (T, k) donde k es el número de salidas
    @param tipo: Tipo de salida de la red neurnal (in ['lineal', 'logistica', 'softmax'])
    @param capas_ocultas: Número de capas ocultas
    @param neuronas_en_capas_ocultas: Número de neuronas en cada capa oculta
    @param alpha: Paso de aprendizaje, por default 0.1
    @param gamma: Factor de olvido para el computo de la inercia 
    @param inter_epoch: Máximo número de epoch (iteraciones) antes de checar por el sobreaprendizaje por default 100
    @param max_epoch: Máximo número de epoch (iteraciones) por default 1000
    @param porcentaje: Porcentaje de datos usados para aprendizaje (el resto para validación) por default 0.8 (80%).
    @param criterio: Minimo valor para considerar sobreaprendizaje (default 0.1)
    @param reinicios: Número de reinicios aleatorios
    
    return: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    
    """
    mejor = 1e20
    mu, std = obtiene_medias_desviaciones(x)
    xn = normaliza(x, mu, std)
    
    for _ in range(reinicios):
        rn = inicializa_red_neuronal(2 + capas_ocutas, 
                                     [x.shape[1]] + 
                                      capas_ocultas * [neuronas_en_capas_ocultas] + 
                                      [y.shape[1]], 
                                     tipo)
        rn['medias'] = mu
        rn['std'] = std
        
        early_stop(xn, y, rn, alpha, gamma, inter_epoch, max_epoch, porcentaje, criterio)
        y_est = feedforward(xn, rn)[-1].T
        actual = costo(y, y_est, rn['tipo'])
        if actual < mejor:
            red_neuronal = deepcopy(rn)
            mejor = actual
    return red_neuronal


def predice(x, red_neuronal):
    """
    Devuelve la prediccion de una red neuronal, solo una función envolvente de feedforward

    @param x: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal`

    """
    xn = normaliza(x, red_neuronal['medias'], red_neuronal['std'])
    y_est = feedforward(xn, red_neuronal)[-1].T
    if red_neuronal['tipo'] in ['logistica', 'softmax']:
        t = np.zeros_like(y_est)
        t[np.arange(y_est.shape[0]), y_est.argmax(axis=1)] = 1
        return t        
    return y_est 
    
            

Y ahora probamos la red neuronal con el problema de los dígitos. en este caso, lo interesante es escoger los parámetros para obtener resultados decentes en un tiempo decente. 

#### Ejercicio 8 Completa el código para la predicción en el problema de los digitos (10 puntos).

In [ ]:
data = np.load("digitos.npz")

x = data['X_entrena']
y = data['T_entrena']
x_prueba = data['X_valida']
y_prueba = data['T_valida']

# Aqui hay que escoger los parámetros de la red
# -----------------------------------------------
tipo = 
capas_ocultas =  
neuronas_en_capas_ocultas = 
alpha = 
gamma = 
inter_epoch = 
max_epoch = 
porcentaje = 
criterio = 
reinicios = 
# ------------------------------------------------

# Esto puede llegar a tardar MUCHO
red_neuronal = aprende(x, y, tipo, capas_ocultas, neuronas_en_capas_ocultas,
            alpha, gamma, inter_epoch, max_epoch, porcentaje, criterio, reinicios)

y_est = predice(x_prueba, red_neuronal)

correctos = np.where(np.abs((y_prueba - y_est)).sum(axis=1) == 0, 1, 0).mean()

print "El porcentaje de valores bien clasificados en el conjunto de prueba es de %2.2f %%" %(100 * correctos)


